<h2>Machine Learning</h2>
<br>
<p>Model learning and predicting</p>

In [43]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder ## might not scale, less effective in regression and tree based algorithms
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
#-------metrics:
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
#-------plots:
from matplotlib import pyplot as plt

In [17]:
df = pd.read_csv('./clean_anime_dataframe')

,Format,Mean Score,Popularity,Favorites,Source,Action,Adventure,Comedy,Drama,Ecchi,...,Mecha,Music,Mystery,Psychological,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Thriller
0,0,66,6050,69,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,72,6307,98,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,60,6176,48,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
3,0,61,43462,656,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,2,76,58959,797,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,2,61,22,1,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9523,5,62,73,2,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9524,4,61,71,2,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9525,5,66,159,2,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


<p>First ,lets split the data to training and testing segments based on the value we want to predict 'Mean Score'</p>

In [13]:
y = df['Mean Score']
X = df.drop('Mean Score', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size= 0.33)

<p>After splitting the data lets create a linear regression model and train it based on the entire dataframe without scaling or taking out unnecessary data</p>

In [19]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [32]:
predict_val = lr_model.predict(X_test)

In [33]:
r2_score(y_test, predict_val)

0.2470620372501885

<p>As you can see, the first model was unsuccessful with an accuracy of 24%, lets try and improve it</p>

In [ ]:
df['PopularityXFaves'] = df['Popularity'] * df['Favorites']